In [ ]:
import os.path
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [ ]:
# import names and emails of all people signed up 
signed_up = pd.read_csv('signed_up.csv')

In [ ]:
# import names of people who are not taking part this time
away = pd.read_csv('away.csv')

In [ ]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [ ]:
class neurocupid:
    """A class to create and keep track of monthly 1-1 neurocupid matches."""    
    def __init__(self, names):
        self.names = sorted(names)
        self.away = []
        self.get_matches()
        self.pairs = []
        self.calc_prob()
    
    def get_matches(self):
        """Gets past matches saved in csv file. If file doesn't exist creates new matrix."""
        dname = globals()['_dh'][0]
        fname = os.path.join(dname,'matches.csv')
        if os.path.isfile(fname):
            self.matches = np.loadtxt(fname, delimiter=',')
        else:
            self.matches = np.ones((len(self.names), len(self.names)))
            
    def calc_prob(self, ignore_away = False, ignore_pairs = False):
        """Calculate pairing probabilities based on past matches."""
        self.prob = 1/self.matches
        # eliminate diagnoal (person can't be paired with themselves)
        np.fill_diagonal(self.prob,0)
        if not ignore_away:
            # eliminate absent people
            i = [self.names.index(name) for name in self.away]
            self.prob[:,i] = 0
        if not ignore_pairs:
            # eliminate already paired people
            i = [self.names.index(name) for name in flatten(self.pairs)]
            self.prob[:,i] = 0
        self.prob = normalize(self.prob, axis=1, norm='l1')
    
    def save_matches(self):
        """Saves updated matches to csv file in directory of notebook."""
        dname = globals()['_dh'][0]
        fname = os.path.join(dname,'matches.csv')
        np.savetxt(fname, self.matches, delimiter=',')
    
    def add_match(self, name1, name2):
        """Adds a match to the matrix."""
        i1 = self.names.index(name1)
        i2 = self.names.index(name2)
        self.matches[i1,i2] = self.matches[i1,i2] + 1
        self.matches[i2,i1] = self.matches[i2,i1] + 1
    
    def remove_match(self, name1, name2):
        """Removes a match from the matrix."""
        i1 = self.names.index(name1)
        i2 = self.names.index(name2)
        self.matches[i1,i2] = self.matches[i1,i2] - 1
        self.matches[i2,i1] = self.matches[i2,i1] - 1
        
    def create_match(self, name):
        """Sample a new match according to probabilities."""
        i = self.names.index(name)
        if sum(self.prob[i]) == 0: # assign to random other pair
            matchi = np.random.choice(len(self.pairs))
            self.add_match(name, self.pairs[matchi][0])
            self.add_match(name, self.pairs[matchi][1])
            self.pairs[matchi].append(name)
        else: # sample match
            match = np.random.choice(self.names,  p=self.prob[i])
            self.add_match(name, match)
            self.pairs.append([name, match])
            self.calc_prob()
    
    def create_round(self, away):
        """Create matches for one round."""
        self.away = sorted(away)
        self.att = list(set(self.names) - set(self.away))
        random.seed()
        random.shuffle(self.att)
        self.pairs = []
        self.calc_prob()
        for n in self.att:
            if n not in flatten(self.pairs):
                self.create_match(n)
        
    def print_pairs(self):
        """Print pairs."""
        text = ''
        for pair in self.pairs:
            text += '\n\n'+'\n'.join(pair)+'\n\n--'
        print(text)
    
    def plot_prob(self):
        """Plot matrix of probabilities of future matches."""
        self.calc_prob(ignore_away = True, ignore_pairs = True)
        fig, ax = plt.subplots(1,1,figsize=(9,9))
        ax = sns.heatmap(self.prob.round(2), annot=True, annot_kws={"size": 7}, vmin=0, vmax=1)
        ax.set_xticklabels(self.names)
        ax.set_yticklabels(self.names)
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
    
    def add_name(self, name):
        """Adds a new member to the matrix."""
        # update names
        self.names.append(name)
        self.names = sorted(self.names)
        i = self.names.index(name)
        # add row and column with 1s into matrix 
        self.matches = np.insert(self.matches, i, 1, axis=0)
        self.matches = np.insert(self.matches, i, 1, axis=1)
    
    def remove_name(self, name):
        """Removes member from the matrix. (Forgets all past matches.)"""
        i = self.names.index(name)
        # update names
        self.names.remove(name)
        # remove row and column from matrix 
        self.matches = np.delete(self.matches, i, axis=0)
        self.matches = np.delete(self.matches, i, axis=1)

In [ ]:
ecn = neurocupid(signed_up.names)

In [ ]:
ecn.plot_prob()

In [ ]:
ecn.create_round(away.names)

In [ ]:
ecn.print_pairs()

In [ ]:
ecn.plot_prob()

In [ ]:
ecn.save_matches()